# Formation : Computer Vision with Python
## Séance 5 : Image classification with CNN
### Auteur : Nennouche Mohamed
### Date : 28/02/2022
### Contenu du notebook :
On va au cours de cette séance introduire l'utilisation de Tensorflow dans les problème de Computer Vision sur des problématiques de classification utilisant des CNN. Donc on fera dans ce notebook :  
- Introduction à Tensorflow 
- Comment créer un modèle avec Tensorflow
    - Méthode séquentielle
    - Méthode fonctionnelle
    - Méthode avec les classes
- Comment entrainer un modèle
- Comment évaluer un modèle
- Comment bien préparer les données pour les traiter avec Tensorflow

Pour se faire on va utiliser une partie du dataset [GTSRB](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign) pour la classification de différents types de plaques de signalisation routière

In [1]:
import tensorflow as tf

import utils

from tensorflow.keras.layers import Conv2D, Input, Dense, MaxPool2D, BatchNormalization, GlobalAvgPool2D, Flatten
from tensorflow.keras import Model

# for callback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

## 1. Préparation des données

### a) Split entraînement - validation

On commence par créer une fonction qui permet de splitter les données d'entraînement en deux dossiers : train et val, le premier pour l'entrainement du modèle et le deuxième pour sa validation (durant l'entraînement)

In [9]:
path_to_data = './Train'
path_to_save_train = './Train_val/train'
path_to_save_val = './Train_val/val'
utils.split_data(path_to_data, path_to_save_train, path_to_save_val)

### b) Préparation des données de test

In [2]:
path_to_images = './Test2'
path_to_csv = './Test.csv'
utils.order_test_set(path_to_images, path_to_csv)

## 2. Mise en place du modèle

### a) Informations préliminaires

On va utiliser GlobalAvgPool2D au lieu de Flatten, permettant de gagner beaucoup de paramètres lors de l'entraînement. Voilà un exemple d'utilisation de GlobalAvgPool2D

In [3]:
input_shape = (2, 4, 5, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.GlobalAveragePooling2D()(x)
print(x.shape)
print(y.shape)

(2, 4, 5, 3)
(2, 3)


In [7]:
print(x)

tf.Tensor(
[[[[ 1.8397477  -1.3505247  -1.5800402 ]
   [ 0.63768935 -1.1999903   0.38274378]
   [-1.0280094  -1.579934    1.624443  ]
   [-0.9099453   0.20088933  1.7756923 ]
   [-1.8025535  -0.91793525  1.7371365 ]]

  [[-0.80055195  0.68446356  1.5627073 ]
   [ 1.2695603  -0.3232048  -0.959329  ]
   [-0.8166272  -1.9070863  -1.3277104 ]
   [ 0.4981948   1.992311    1.6657081 ]
   [ 0.4804979  -0.10094799 -0.90984946]]

  [[ 1.8846945   1.3698827  -0.44374925]
   [-0.82751834  1.2349504  -0.42769873]
   [-0.38052356  0.9910699  -0.22697818]
   [-1.5485541   0.68284845 -0.19728404]
   [-0.34907854 -0.4985748   0.9208069 ]]

  [[-1.48777    -0.18260342 -0.2571184 ]
   [-1.5337881  -1.17411     1.2933922 ]
   [ 0.46214715 -0.6052504  -0.18489166]
   [ 1.1469971   0.42211697 -0.4337584 ]
   [ 0.93902516 -0.3441186   0.20144875]]]


 [[[-1.1927199   1.805365    1.140721  ]
   [ 0.22117287  0.22867215  0.18312322]
   [ 0.6205142   0.46545762 -1.2988738 ]
   [-0.8268708  -0.20306446  0.05548

In [8]:
print(y)

tf.Tensor(
[[-0.11631831 -0.13028741  0.21078357]
 [-0.13969752  0.23934543  0.0197091 ]], shape=(2, 3), dtype=float32)


### b) Création du modèle

In [4]:
def streetsigns_model(nbr_classes) :
    my_input = Input(shape=(60,60, 3)) # taille moyenne et les 3 canaux RGB
    x= Conv2D(32, (3,3), activation='relu')(my_input)
    x= Conv2D(64, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= Conv2D(128, (3,3), activation='relu')(x)
    x= MaxPool2D()(x)
    x= BatchNormalization()(x)

    x= GlobalAvgPool2D()(x) # On moyenne la totalité de la matrice
    #x = Flatten()(x)
    x= Dense(64, activation='relu')(x)
    x= Dense(nbr_classes, activation='softmax')(x)
    model = Model(inputs=my_input, outputs=x)
    return model

In [5]:
model = streetsigns_model(43)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 58, 58, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 128)       73856 

## 3. Chargement des données

### Générateur de données 

Pour alimenter notre modèle pour l'entrainement et ensuite le test, il contiendra : 
- préprocessing de toutes les images
- préparation des images pour l'entrainement 
- acheminement des images

In [6]:
train_data_path = './Train_val/train'
val_data_path = './Train_val/val'
test_data_path = './Test2'
batch_size = 64

train_generator, val_generator, test_generator= utils.create_generators(batch_size, train_data_path, val_data_path, test_data_path)

nbr_classes = train_generator.num_classes

Found 35288 images belonging to 43 classes.
Found 3921 images belonging to 43 classes.
Found 12630 images belonging to 43 classes.


In [7]:
model = streetsigns_model(nbr_classes)

## 4. Entrainement et fitting

### a) Préparation des callbacks

In [8]:
# callbacks
path_to_save_model = './Models' # ou sauvegarder le modèle
ckpt_saver = ModelCheckpoint(
    path_to_save_model,
    monitor='val_accuracy', # sur quoi on se base pour voir le meilleur
    mode = 'max', # max de l'accuracy sur la validation
    save_best_only = True,
    save_freq='epoch', # ne voit qu'à la fin de l'époque
    verbose=1
) 

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10 # après 10 époques ca change pas on s'arrête
)

### b) Compilation du modèle

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # on choisit categorical_crossentropy car dans les générateurs on a défini categorical comme class_mode

Définition du nombre d'époques

In [10]:
epochs = 15

### c) Entraînement

In [11]:
# Le générateur contient autant les images que les labels
model.fit(
    train_generator,
    epochs = epochs,
    batch_size = batch_size,
    validation_data = val_generator,
    callbacks=[ckpt_saver, early_stop]
)

Epoch 1/15
552/552 [==============================] - ETA: 0s - loss: 2.4154 - accuracy: 0.3227
Epoch 1: val_accuracy improved from -inf to 0.18490, saving model to .\Models


INFO:tensorflow:Assets written to: .\Models\assets


INFO:tensorflow:Assets written to: .\Models\assets


552/552 [==============================] - 828s 1s/step - loss: 2.4154 - accuracy: 0.3227 - val_loss: 3.1183 - val_accuracy: 0.1849
Epoch 2/15
552/552 [==============================] - ETA: 0s - loss: 1.1370 - accuracy: 0.6674
Epoch 2: val_accuracy improved from 0.18490 to 0.60367, saving model to .\Models


INFO:tensorflow:Assets written to: .\Models\assets


...

INFO:tensorflow:Assets written to: .\Models\assets


552/552 [==============================] - 464s 841ms/step - loss: 0.0265 - accuracy: 0.9934 - val_loss: 0.0676 - val_accuracy: 0.9804
Epoch 13/15
552/552 [==============================] - ETA: 0s - loss: 0.0339 - accuracy: 0.9900
Epoch 13: val_accuracy did not improve from 0.98036
552/552 [==============================] - 450s 815ms/step - loss: 0.0339 - accuracy: 0.9900 - val_loss: 0.1493 - val_accuracy: 0.9541
Epoch 14/15
552/552 [==============================] - ETA: 0s - loss: 0.0183 - accuracy: 0.9953
Epoch 14: val_accuracy improved from 0.98036 to 0.99107, saving model to .\Models


INFO:tensorflow:Assets written to: .\Models\assets


INFO:tensorflow:Assets written to: .\Models\assets


552/552 [==============================] - 455s 825ms/step - loss: 0.0183 - accuracy: 0.9953 - val_loss: 0.0340 - val_accuracy: 0.9911
Epoch 15/15
552/552 [==============================] - ETA: 0s - loss: 0.0132 - accuracy: 0.9969
Epoch 15: val_accuracy did not improve from 0.99107
552/552 [==============================] - 468s 848ms/step - loss: 0.0132 - accuracy: 0.9969 - val_loss: 0.0520 - val_accuracy: 0.9867


## 5. Chargement et évaluation du modèle

### a) Chargement du modèle

In [12]:
model = tf.keras.models.load_model('./Models')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 58, 58, 32)        896       
                                                                 
 conv2d_4 (Conv2D)           (None, 56, 56, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 128)       7385

### b) Evaluation du modèle

In [13]:
model.evaluate(test_generator, batch_size=64)

198/198 [==============================] - 125s 633ms/step - loss: 0.3018 - accuracy: 0.9289


[0.3017587661743164, 0.9288994669914246]

### c) Comment améliorer le modèle ?

Pour améliorer le modèle on a plusieurs possibilités : 
- Changer la taille du Batch
- Augmenter (ou diminuer) le nombre d'époques
- Changer l'architecture du modèle (changer les couches ou en ajouter ou diminuer)
- Dans la création du générateur (dans la partie ImageDataGenerator) il y a un certain nombre de techniques pour de la data augmentation qu'on peut tenter.
- On peut mettre en place plusieurs pré-processeurs pour les adapter, chacun à une partie du problème (train et pas validation et test par exemple) surtout dans le cas de l'augmentation des données (avec des shifts et des zoom) 
- On peut changer l'optimizer en utilisant opitmizer = tf.keras.optimizers.NomOptimizer() et on choisit d'après la documentation qu'on a
- Ajouter et changer le learning rate et le momentum et l'ajouter à l'optimizer

### d) Test du modèle à part entière

In [15]:
img_path = "./Test2/0/00403.png"
model = tf.keras.models.load_model('./Models')
prediction = utils.predict_with_model(model, img_path)
print(f'La classe est {prediction}')

1/1 [==============================] - 0s 149ms/step
La classe est 0
